Splunk MLToolkit example of "Predict Server Power Consumption" using pyspark mllib

In [3]:
#pixdust speaks spark
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import pixiedust
randn = np.random.randn

In [4]:
#get the url for the spark ui
sc

<SparkContext master=local[*] appName=PySparkShell>

In [5]:
import glob
filenames = glob.glob("data/server_power.csv")
server_power = pd.concat( [ pd.read_csv(f) for f in filenames ] )
server_power.describe()

,ac_power,total-unhalted_core_cycles,total-instructions_retired,total-last_level_cache_references,total-memory_bus_transactions,total-cpu-utilization,total-disk-accesses,total-disk-blocks,total-disk-utilization
count,31272.000000,3.127200e+04,3.127200e+04,3.127200e+04,3.127200e+04,31271.000000,31272.000000,31272.000000,31272.000000
mean,239.802467,2.309277e+09,2.714148e+09,1.443819e+08,5.145059e+06,97.564742,6.844126,193.496255,5.247245
std,24.206970,3.261582e+09,3.810763e+09,2.164751e+08,9.314938e+06,135.762684,24.023074,1130.485155,20.846893
min,218.000000,4.168413e+06,3.708610e+06,6.719500e+04,2.789000e+03,0.000000,0.000000,0.000000,0.000000
25%,225.000000,5.144853e+08,5.600434e+08,2.277050e+07,9.552775e+04,27.190000,0.000000,0.000000,0.000000
50%,230.000000,1.076023e+09,1.374843e+09,6.006047e+07,1.073730e+06,52.640000,0.000000,0.000000,0.000000
75%,242.000000,2.335011e+09,2.606795e+09,1.359125e+08,5.454786e+06,99.840000,8.410000,72.000000,5.250000
max,336.000000,1.861924e+10,3.799910e+10,1.172397e+09,5.672846e+07,800.000000,822.920000,18602.530000,252.160000


In [6]:
display(server_power)

_time,ac_power,total-unhalted_core_cycles,total-instructions_retired,total-last_level_cache_references,total-memory_bus_transactions,total-cpu-utilization,total-disk-accesses,total-disk-blocks,total-disk-utilization
00:14:04 2008-Apr-23,234.0,993265104.0,2052637320.0,64034618.0,56273.0,39.55,0.0,0.0,0.0
02:51:07 2008-Apr-23,227.0,810867322.0,1642382151.0,52583034.0,114311.0,58.86,19.0,192.0,14.8
02:54:13 2008-Apr-23,227.0,809307702.0,1642590132.0,52728389.0,100259.0,57.95,0.0,0.0,0.0
13:53:20 2008-Apr-14,220.0,82323898.0,52462821.0,285004.0,139916.0,0.99,0.0,0.0,0.0
04:55:55 2008-Apr-23,238.0,2096383106.0,1994638742.0,138864454.0,6447297.0,79.87,0.0,0.0,0.0
04:30:02 2008-Apr-23,238.0,2161886980.0,1922625283.0,142212755.0,6386356.0,90.01,14.29,163.27,7.35
00:32:07 2008-Apr-23,304.0,10898743125.0,10379176772.0,769029061.0,23977181.0,459.15,5.15,41.24,4.54
23:19:44 2008-Apr-22,223.0,459874954.0,838689904.0,24232020.0,23026.0,27.27,0.0,0.0,0.0
20:49:08 2008-Apr-22,225.0,496693355.0,496900124.0,34417213.0,1284022.0,21.68,0.0,0.0,0.0
04:30:25 2008-Apr-23,236.0,2300095428.0,1990820022.0,147285431.0,6628731.0,87.92,0.0,0.0,0.0


In [7]:
#data preperation
from pyspark.ml.regression import LinearRegression  
from pyspark.ml.feature import VectorAssembler  
from pyspark.ml.feature import StandardScaler  
data_sdf = spark.createDataFrame(server_power)

data_sdf = data_sdf \
    .withColumn('label',data_sdf.ac_power) 
    
features = ["total-unhalted_core_cycles", "total-instructions_retired", "total-last_level_cache_references", "total-memory_bus_transactions", "total-cpu-utilization", "total-disk-accesses", "total-disk-blocks", "total-disk-utilization"]
label = ["label"]  

model_data_sdf = data_sdf.select("label","ac_power","_time",*features)  
(training_data, test_data) = model_data_sdf.randomSplit([.7, .3])

In [8]:
#model_data_sdf.show()
model_data_sdf.printSchema()

root
 |-- label: double (nullable = true)
 |-- ac_power: double (nullable = true)
 |-- _time: string (nullable = true)
 |-- total-unhalted_core_cycles: double (nullable = true)
 |-- total-instructions_retired: double (nullable = true)
 |-- total-last_level_cache_references: double (nullable = true)
 |-- total-memory_bus_transactions: double (nullable = true)
 |-- total-cpu-utilization: double (nullable = true)
 |-- total-disk-accesses: double (nullable = true)
 |-- total-disk-blocks: double (nullable = true)
 |-- total-disk-utilization: double (nullable = true)



In [9]:
#machine learning library
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel 
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

#indexer = StringIndexer(inputCol="_time", outputCol="_time_Index")
##assembler = VectorAssembler(inputCols=[*features], outputCol="features")
##assembler = VectorAssembler(inputCols=["total-cpu-utilization","total-disk-utilization"], outputCol="features")
assembler = VectorAssembler(inputCols=["total-cpu-utilization"], outputCol="features")
lr = LinearRegression(labelCol="label", featuresCol="features", predictionCol="prediction_ac_power",maxIter=10, regParam=0.3, elasticNetParam=0.8)

#pipeline = Pipeline(stages=[indexer, assembler, lr])
pipeline = Pipeline(stages=[assembler, lr])

model = pipeline.fit(training_data)
prediction = model.transform(training_data)
#prediction.show()
#display(prediction)

In [25]:
#prediction.show()
display(prediction)

label,ac_power,_time,total-unhalted_core_cycles,total-instructions_retired,total-last_level_cache_references,total-memory_bus_transactions,total-cpu-utilization,total-disk-accesses,total-disk-blocks,total-disk-utilization,features,prediction_ac_power
220.0,220.0,13:53:32 2008-Apr-14,4408034.0,3807604.0,73329.0,3629.0,0.0,0.0,0.0,0.0,[0.0],223.251425290501
220.0,220.0,13:54:03 2008-Apr-14,4848740.0,3942322.0,84238.0,5292.0,0.0,1.23,9.82,0.98,[0.0],223.251425290501
221.0,221.0,13:35:20 2008-Apr-15,7881505.0,4837905.0,129524.0,16231.0,0.0,0.0,0.0,0.0,[0.0],223.251425290501
221.0,221.0,13:52:42 2008-Apr-14,6477011.0,7257153.0,97290.0,32586.0,0.0,0.0,0.0,0.0,[0.0],223.251425290501
222.0,222.0,12:52:22 2008-Apr-15,81270981.0,10438759.0,419842.0,559271.0,0.0,0.0,0.0,0.0,[0.0],223.251425290501
222.0,222.0,13:12:53 2008-Apr-15,81178282.0,10582754.0,428127.0,534567.0,0.0,0.0,0.0,0.0,[0.0],223.251425290501
222.0,222.0,13:23:33 2008-Apr-15,6366483.0,4383417.0,107103.0,11051.0,0.99,14.29,204.08,8.57,[0.99],223.41914873271418
222.0,222.0,13:24:45 2008-Apr-15,11809338.0,6327950.0,243515.0,43111.0,-0.0,47.87,2331.91,30.21,[0.0],223.251425290501
222.0,222.0,13:25:41 2008-Apr-15,25130308.0,11688935.0,559665.0,121029.0,-0.0,122.0,7184.0,99.2,[0.0],223.251425290501
222.0,222.0,13:27:05 2008-Apr-15,7221924.0,4959921.0,121703.0,11926.0,0.0,0.0,0.0,0.0,[0.0],223.251425290501


In [11]:
#summary information
ts = model.stages[1].summary
##dir(ts)
[ts.r2, ts.meanAbsoluteError, ts.meanSquaredError, ts.rootMeanSquaredError]
##[0.9484640313770627, 3.415719514221418, 30.493298690749704, 5.522073767231809]

[0.9341680067127939, 4.274187433645905, 39.486923644724705, 6.283862159908085]

In [12]:
#save/load model for another time
#model.write().overwrite().save("sample-model")
#pipeline.write().overwrite().save("sample-pipeline")
#pipeline_in = Pipeline.read().load("sample-pipeline")
#model_in = PipelineModel.read().load("sample-model")

In [13]:
ts.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -4.251425290501004|
| -4.251425290501004|
| -4.251425290501004|
| -4.251425290501004|
| -4.251425290501004|
| -4.251425290501004|
| -4.251425290501004|
| -4.251425290501004|
| -4.419148732714177|
| -3.751425290501004|
| -3.751425290501004|
|-3.4174545565302026|
| -3.251425290501004|
| -3.251425290501004|
|-3.4377846707378694|
| -3.251425290501004|
|-3.4462555516577424|
| -3.419148732714177|
| -3.251425290501004|
| -3.419148732714177|
+-------------------+
only showing top 20 rows



In [14]:
pdf = ts.residuals.toPandas()
pdf.insert(0, 'ID', range(0, len(pdf)))

In [15]:
display(pdf)

In [24]:
#with spark sql
prediction.createOrReplaceTempView("prediction")
sqlDF = spark.sql("SELECT monotonically_increasing_id() as id, (ac_power - prediction_ac_power) as residuals  FROM prediction")
#sqlDF.show()
display(sqlDF)

In [35]:
#with native manipulation
ts.residuals.agg({"residuals": "avg"}).show()

+--------------------+
|      avg(residuals)|
+--------------------+
|-2.82418086918756...|
+--------------------+

